A few guidelines:

    The tasks can be solved in various ways; do it in a way that will show off your skills.
    Feel free to ask for clarifications
    The deadline for submission is two weeks or sooner
    The tasks can be solved in a variety of ways, do it in a way that will show off your skills.
    Feel free to make reasonable assumptions or ask clarifying questions
    Deliverable is a Github repo and a direct web link to try the solution.
    You have one chance to leave first impression - try to build something that is intuitive, easy to run and has readable code

Thank you for participating. Good luck, and have fun!
Warmup task - Reranking Hacker News

Build an API that given user submitted bio, returns top 500 stories from Hacker News front page (use their API for this), ranked in the order of relevancy to user's interests.

Example input:
I am a theoretical biologist, interested in disease ecology. My tools are R, clojure , compartmentalism disease modeling, and statistical GAM models, using a variety of data layers (geophysical, reconstructions, climate, biodiversity, land use). Besides that I am interested in tech applied to the a subset of the current problems of the world (agriculture / biodiversity / conservation / forecasting), development of third world countries and AI, large language models.

Output:
Top 500 items from Hacker News, ranked in the order of relevance to the user.

The solution should optimize for development time - what is the simplest/quickest way to do this that you can think of? Spend no more than 4 hours on this.
Deliverables

    GitHub Repo
    Link to a live demo containing a text area for the input and a button to submit. The results will be shown below.
    Explain your thought process and decisions you made during creation of the solution.

# Database Setup:

In [1]:
import sqlite3
db = sqlite3.connect("hackernews.db")

In [2]:
from dataclasses import asdict, dataclass

@dataclass
class HNStory:
    submitter: str
    descendant_count: int
    story_id: int
    score: int
    time: int
    title: str
    url: str

    @classmethod
    def from_api_json(cls, json):
        json['submitter'] = json.pop('by')
        json['story_id'] = json.pop('id')
        json['descendant_count'] = json.pop('descendants')
        try:
            json.pop('kids')
        except:
            pass
        try:
            json.pop('type')
        except:
            pass
        return cls(**json)

db.execute("CREATE TABLE IF NOT EXISTS hn(story_id INTEGER PRIMARY KEY, title TEXT, url TEXT, time INTEGER, submitter TEXT, score INTEGER, descendant_count INTEGER);")
db.row_factory = sqlite3.Row  # So we get back dicts.

# If we set the row factory to a dict factory like this then we can automatically get back stories, but it will cause problems when we filter:
#def dict_factory(cursor, row):
#    fields = [column[0] for column in cursor.description]
#    return HNStory(**{key: value for key, value in zip(fields, row)})
#db.row_factory = dict_factory
        
# If we want to do a fulltext search later we can do this, but it won't work on MacOS.
# By default, MacOS will block extensions.  Make sure you've got a PyEnv or non-OS Build if you see "AttributeError: 'sqlite3.Connection' object has no attribute 'enable_load_extension'"
#db.enable_load_extension(True)
#db.execute("select load_extension('./fts3.so')") OR con.load_extension("./fts3.so")
#db.enable_load_extension(False)
#db.execute("CREATE VIRTUAL TABLE recipe USING fts3(name, ingredients)")
#db.executescript("""INSERT INTO recipe (name, ingredients) VALUES('broccoli stew', 'broccoli peppers cheese tomatoes'); INSERT INTO recipe (name, ingredients) VALUES('pumpkin stew', 'pumpkin onions garlic celery');""")
#for row in con.execute("SELECT rowid, name, ingredients FROM recipe WHERE name MATCH 'pie'"):
# See https://docs.python.org/3/library/sqlite3.html#sqlite3.Connection.enable_load_extension

# For SQLite Vec: https://github.com/asg017/sqlite-vec

# Data Ingest:

In [3]:
# Pulling HN results:
import requests
import time
from datetime import datetime, timedelta

# We could do this async, but I don't want to hammer the server.  We can get rate limited pretty easily.
# TODO: check for stories we've already fetched.
# TODO: reuse connections so we're not bombarding HN with lots of separate connection opens.
# TODO: use async to pull the requests in parallel, limiting the number of workers.
def fetch_hn(story_limit: int = 500, rate_limit_seconds: float = 0.1, backoff_failure_seconds: float = 1.0):
    id_request = requests.get("https://hacker-news.firebaseio.com/v0/topstories.json")
    if not id_request.ok:
        raise Exception("Failed to fetch HN results: {out.content}")
    story_ids = id_request.json()
    last_story_fetched = datetime.now() - timedelta(seconds=rate_limit_seconds)
    for story_id in story_ids:
        retry_count = 3
        story_json = None
        while retry_count > 0 and story_json is None:
            try:
                time_since_last_fetch = (datetime.now() - last_story_fetched).total_seconds()
                time.sleep(max(0, rate_limit_seconds - time_since_last_fetch))
                content_request = requests.get(f"https://hacker-news.firebaseio.com/v0/item/{story_id}.json")
                story_json = HNStory.from_api_json(content_request.json())
                last_story_fetched = datetime.now()
            except Exception as e:
                print(f"Failed to grab story id {story_id}: {e}")
                retry_count -= 1
                time.sleep(backoff_failure_seconds)
        if story_json is not None:
            yield story_json
            story_limit -= 1
        if story_limit <= 0:
            break

In [4]:
hn_generator = fetch_hn()
for story in hn_generator:
    print(story)
    break

HNStory(submitter='mfiguiere', descendant_count=56, story_id=43538919, score=115, time=1743449751, title='JEP Draft: Prepare to Make Final Mean Final', url='https://openjdk.org/jeps/8349536')


In [35]:
def refresh_database(connection, hn_generator = None):
    if hn_generator is None:
        hn_generator = fetch_hn()
    for story in hn_generator:
        # If we didn't have to wait on network latency this would be more efficient:
        # cur.executemany("INSERT INTO data VALUES(?)", rows)
        print(f"Fetched {story}")
        #db.execute("INSERT INTO hn VALUES(:story_id, :title, :url, :time, :submitter, :score, :descendant_count)", (asdict(story),))
        db.execute("INSERT INTO hn VALUES(:story_id, :title, :url, :time, :submitter, :score, :descendant_count)", (story.story_id, story.title, story.url, story.time, story.submitter, story.score, story.descendant_count,))
    db.commit()

In [36]:
refresh_database(db)

Fetched HNStory(submitter='mfiguiere', descendant_count=59, story_id=43538919, score=124, time=1743449751, title='JEP Draft: Prepare to Make Final Mean Final', url='https://openjdk.org/jeps/8349536')
Fetched HNStory(submitter='jedeusus', descendant_count=9, story_id=43539585, score=68, time=1743452998, title='Go Optimization Guide', url='https://goperf.dev/')
Fetched HNStory(submitter='robin_reala', descendant_count=66, story_id=43533362, score=503, time=1743417597, title='The demoscene as a UNESCO heritage in Sweden', url='https://www.goto80.com/the-demoscene-as-a-unesco-heritage-in-sweden')
Fetched HNStory(submitter='leotravis10', descendant_count=19, story_id=43538192, score=90, time=1743446137, title="Notes on the Pentium's Microcode Circuitry", url='https://www.righto.com/2025/03/pentium-microcde-rom-circuitry.html')
Fetched HNStory(submitter='bookofjoe', descendant_count=28, story_id=43511529, score=78, time=1743208287, title='The Guardian flourishes without a paywall', url='http

In [37]:
cursor = db.execute("SELECT * FROM hn WHERE score > ? ORDER BY time DESC LIMIT 10;", (0,))
for r in cursor:
    print(dict(r))
    print(HNStory(**dict(r)))

{'story_id': 43539585, 'title': 'Go Optimization Guide', 'url': 'https://goperf.dev/', 'time': 1743452998, 'submitter': 'jedeusus', 'score': 68, 'descendant_count': 9}
HNStory(submitter='jedeusus', descendant_count=9, story_id=43539585, score=68, time=1743452998, title='Go Optimization Guide', url='https://goperf.dev/')
{'story_id': 43538919, 'title': 'JEP Draft: Prepare to Make Final Mean Final', 'url': 'https://openjdk.org/jeps/8349536', 'time': 1743449751, 'submitter': 'mfiguiere', 'score': 124, 'descendant_count': 59}
HNStory(submitter='mfiguiere', descendant_count=59, story_id=43538919, score=124, time=1743449751, title='JEP Draft: Prepare to Make Final Mean Final', url='https://openjdk.org/jeps/8349536')
{'story_id': 43538192, 'title': "Notes on the Pentium's Microcode Circuitry", 'url': 'https://www.righto.com/2025/03/pentium-microcde-rom-circuitry.html', 'time': 1743446137, 'submitter': 'leotravis10', 'score': 90, 'descendant_count': 19}
HNStory(submitter='leotravis10', descend

In [ ]:
db.close()

# Text Embedding and Retrieval

In [ ]:
%pip install sqlite-vec

In [ ]:
%pip install sentence-transformers

In [14]:
import sqlite_vec
db.enable_load_extension(True)
sqlite_vec.load(db)
db.enable_load_extension(False)

In [15]:
import numpy
from sentence_transformers import SentenceTransformer

# NOTE:
# If we convert embeddings into numpy arrays we don't need to use the serialize_float32 method here:
#from sqlite_vec import serialize_float32
#result = db.execute('select vec_length(?)', [serialize_float32([0.1, 0.2, 0.3, 0.4])])

# Love Matryoshka embeddings: https://huggingface.co/blog/matryoshka
EMBEDDING_DIM = 64
model = SentenceTransformer('tomaarsen/mpnet-base-nli-matryoshka', truncate_dim=EMBEDDING_DIM)
embeddings = model.encode(["Test, 1, 2, 3.", "This is only a test.", "This is a test."])
for r in db.execute("SELECT vec_length(?)", [embeddings[0].astype(numpy.float32)]):
    print(dict(r))

E:\Applications\miniconda3\envs\default\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


{'vec_length(?)': 64}


In [38]:
# Create a new table for the embeddings so we can search and join.
print(dict(db.execute(f"CREATE VIRTUAL TABLE IF NOT EXISTS story_embeddings USING vec0(embedding float[{EMBEDDING_DIM}])")))
db.commit()

OperationalError: table story_embeddings already exists

In [39]:
[dict(r) for r in db.execute("SELECT story_id, TITLE FROM hn WHERE story_id NOT IN (SELECT rowid FROM story_embeddings) LIMIT 10")]

[{'story_id': 43511529, 'title': 'The Guardian flourishes without a paywall'},
 {'story_id': 43515622,
  'title': 'Show HN: I made a little puzzle game about a rogue chess knight'},
 {'story_id': 43533362,
  'title': 'The demoscene as a UNESCO heritage in Sweden'},
 {'story_id': 43535688, 'title': 'It’s not mold, it’s calcium lactate (2018)'},
 {'story_id': 43535943, 'title': 'Turso SQLite Offline Sync Public Beta'},
 {'story_id': 43536146,
  'title': 'MLB says Yankees\' new "torpedo bats" are legal and likely coming'},
 {'story_id': 43538113,
  'title': 'Honey has now lost 4M Chrome users after shady tactics were revealed'},
 {'story_id': 43538192, 'title': "Notes on the Pentium's Microcode Circuitry"},
 {'story_id': 43538919,
  'title': 'JEP Draft: Prepare to Make Final Mean Final'},
 {'story_id': 43539585, 'title': 'Go Optimization Guide'}]

In [41]:
# Populate all the stories
with db:
    # We should do this in batches.
    results_processed = 1
    while results_processed > 0:
        cursor = db.execute("SELECT story_id, title FROM hn WHERE story_id NOT IN (SELECT rowid FROM story_embeddings) LIMIT 10")
        batch_ids = list()
        batch_headlines = list()
        batch_to_insert = list()
        for row in cursor:
            batch_ids.append(row['story_id'])
            batch_headlines.append(row['title'])
        embeddings = model.encode(batch_headlines)
        for idx, story_id in enumerate(batch_ids):
            batch_to_insert.append((story_id, embeddings[idx].astype(numpy.float32)))
        db.executemany("INSERT INTO story_embeddings(rowid, embedding) VALUES (?, ?)", batch_to_insert)
        results_processed = len(batch_to_insert)
    db.commit()

"""
items = [
    (1, [0.1, 0.1, 0.1, 0.1]),
    (2, [0.2, 0.2, 0.2, 0.2]),
    (3, [0.3, 0.3, 0.3, 0.3]),
    (4, [0.4, 0.4, 0.4, 0.4]),
    (5, [0.5, 0.5, 0.5, 0.5]),
]
query = [0.3, 0.3, 0.3, 0.3]
"""

In [62]:
import struct
def deserialize_float32(vector, dimensionality: int):
    return struct.unpack("%sf" % dimensionality, vector)

for r in db.execute("SELECT * FROM story_embeddings LIMIT 5"):
    print(r['rowid'])
    print(deserialize_float32(r['embedding'], EMBEDDING_DIM))
    break

43511529
(0.09066823869943619, 0.17725828289985657, -0.005500168539583683, -0.10034725815057755, 0.0164562426507473, 0.06880630552768707, 0.12393510341644287, -0.18441611528396606, 0.12493689358234406, -0.06061761826276779, 0.13532033562660217, 0.015297234058380127, -0.15693846344947815, 0.1349973976612091, -0.1638118326663971, 0.048728156834840775, 0.006766803562641144, 0.035836007446050644, -0.06102864071726799, -0.012957364320755005, -0.2449144423007965, -0.009473470039665699, -0.09948713332414627, -0.202688530087471, 0.31311124563217163, 0.01912880316376686, 0.2371511161327362, -0.12405090034008026, 0.026695838198065758, 0.02410670556128025, -0.2176799327135086, -0.018366370350122452, 0.0028383596800267696, 0.10722251236438751, 0.045345406979322433, -0.03827964514493942, 0.019169138744473457, 0.06108715385198593, -0.0452619306743145, -0.13730375468730927, -0.10532908141613007, 0.05116068571805954, -0.019994452595710754, 0.1372452676296234, -0.03213982656598091, -0.01223244331777095

In [56]:
rows = db.execute(
    """
    SELECT story_id, title FROM hn WHERE story_id IN (
      SELECT
        rowid
      FROM story_embeddings
      WHERE embedding MATCH ?
      ORDER BY distance ASC
      LIMIT 5
    )
    """,
    #[serialize_f32(model(["Embedding models."])[0])],
    ((model.encode(["Microprocessors and embeddings."])[0]),),
).fetchall()
for r in rows:
    print(dict(r))

{'story_id': 43515622, 'title': 'Show HN: I made a little puzzle game about a rogue chess knight'}
{'story_id': 43535688, 'title': 'It’s not mold, it’s calcium lactate (2018)'}
{'story_id': 43538192, 'title': "Notes on the Pentium's Microcode Circuitry"}
{'story_id': 43538919, 'title': 'JEP Draft: Prepare to Make Final Mean Final'}
{'story_id': 43539585, 'title': 'Go Optimization Guide'}


In [65]:
rows = db.execute(
    """
    SELECT hn.story_id, hn.title, story_embeddings.distance 
    FROM hn 
    JOIN story_embeddings on hn.story_id = story_embeddings.rowid 
    WHERE embedding MATCH ? AND k=5
    ORDER BY distance ASC
    LIMIT 5
    """,
    #[serialize_f32(model(["Embedding models."])[0])],
    ((model.encode(["Microprocessors and embeddings."])[0]),),
).fetchall()
for r in rows:
    print(dict(r))

{'story_id': 43538192, 'title': "Notes on the Pentium's Microcode Circuitry", 'distance': 0.7702715992927551}
{'story_id': 43515622, 'title': 'Show HN: I made a little puzzle game about a rogue chess knight', 'distance': 1.230692982673645}
{'story_id': 43535688, 'title': 'It’s not mold, it’s calcium lactate (2018)', 'distance': 1.3278286457061768}
{'story_id': 43539585, 'title': 'Go Optimization Guide', 'distance': 1.3478636741638184}
{'story_id': 43538919, 'title': 'JEP Draft: Prepare to Make Final Mean Final', 'distance': 1.4004002809524536}
